In [1]:
import os

In [12]:
from massageData import runPipeline, readData
from runAutoML import runAutoML, runTPot
from constants import PROCESSED_PATH, RAW_PATH

In [3]:
from sklearn.preprocessing import normalize, OneHotEncoder, LabelBinarizer

In [4]:
datafn = 'HOUR_00003.csv'
datafn = os.path.join(PROCESSED_PATH, datafn)
ids_fn = os.path.join(RAW_PATH, 'd_ids_split.pickle')

In [5]:
df = readData(datafn)
df = df[df.columns[[0,1,4,2,3]+list(range(5,len(df.columns)))]]

In [7]:
featuresCols = df.columns[3:-2]

In [8]:
split_df = runPipeline(df, ids_fn, featuresCols)

test (5579, 205)
devel (16609, 205)
valid (5492, 205)
synthesizing data
test {0: 0.7397383043556193, 1: 0.10503674493636853, 2: 0.05968811614984764, 3: 0.09553683455816454}
ratio {0: 1.0034558556878863, 1: 0.9895378880590725, 2: 0.9095045148007517, 3: 1.0494519081855522}
valid {0: 0.7408958485069191, 1: 0.10342316096139839, 2: 0.06900946831755281, 3: 0.08667152221412965}
ratio {0: 1.0050260656526804, 1: 0.9743365175314044, 2: 1.0515396874185639, 3: 0.9520683283429097}
test 1 586 7
test 2 333 31
valid 1 568 15
valid 3 476 23
test (5586, 102)
test (5617, 102)
valid (5507, 102)
valid (5530, 102)
test 1 593
test 2 364
valid 1 583
valid 3 499


In [14]:
test = split_df['test']
valid = split_df['valid']
devel = split_df['devel']

train = devel.drop(['SUBJECT_ID','HADM_ID','ETHNICITY','TSTAGE', 'P TSTAGE', 'P STAGE'], axis=1)
testv = test.drop(['SUBJECT_ID','HADM_ID','ETHNICITY','TSTAGE', 'P TSTAGE', 'P STAGE'], axis=1)

In [15]:
x_train = train.values[:, :-2]
y_train = train.values[:, -1]
x_train = normalize(x_train, axis=0)
ohe = LabelBinarizer()
ohe.fit(y_train.reshape(-1, 1))

x_test = testv.values[:, :-2]
x_test = normalize(x_test, axis=0)
y_test = testv.values[:, -1]

ohe_y_train = ohe.transform(y_train.reshape(-1,1))
ohe_y_test = ohe.transform(y_test.reshape(-1,1))
print(ohe_y_train.shape)
print(ohe_y_test.shape)

(16609, 4)
(5617, 4)


In [ ]:
# runTraditionalModels(x_train, ohe_y_train, x_test, ohe_y_test, datafn)

In [16]:
aMLScore = runAutoML(train, testv)
print(aMLScore)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.10
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingClassifier to predict STAGE
Started at:
20

/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


The printed list will only contain at most the top 100 features.
+----+-----------------------------------+--------------+----------+-------------------+-------------------+-----------+-----------+-----------+-----------+
|    | Feature Name                      |   Importance |    Delta |   FR_Decrementing |   FR_Incrementing |   FRD_abs |   FRI_abs |   FRD_MAD |   FRI_MAD |
|----+-----------------------------------+--------------+----------+-------------------+-------------------+-----------+-----------+-----------+-----------|
| 55 | P YEAST                           |       0.0000 |   0.0000 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 64 | P BACTERIA                        |       0.0000 |   0.0000 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 85 | P ANISOCYTOSIS                    |       0.0000 |   0.0000 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.

0.8144917215595514


In [ ]:
runTPot(x_train, x_test, y_train, y_test)